In [2]:
from numpy import sqrt, exp, log, pi
from scipy.stats import norm
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import pandas as pd
import datetime as dt
from datetime import date,datetime,timedelta
import warnings
#from scipy.interpolate import griddata
import plotly.graph_objects as go
from arch import arch_model
import nbformat
from matplotlib.colors import Normalize
%matplotlib inline
import plotly.io as pio
pio.renderers.default = "notebook_connected"
warnings.filterwarnings("ignore")

#### Option pricing object

In [4]:
class Option:
    def __init__(self,sigma,S,K,r,t,option_type = "c"):
        self.sigma = sigma
        self.S = S
        self.K = K
        self.r = r
        self.t = t
        self.option_type = option_type
    def _calc_d(self):
        d1 = np.multiply(1. / self.sigma * np.divide(1., np.sqrt(self.t)),\
                        np.log(self.S/self.K) + (self.r + self.sigma**2 / 2.) * self.t)
        d2 = d1 - self.sigma * np.sqrt(self.t)
        return d1,d2
    def option_price(self):
        d1,d2 = self._calc_d()
        if self.option_type == "c":
            return (np.multiply(self.S, norm.cdf(d1)) - \
        np.multiply(norm.cdf(d2) * self.K, np.exp(-self.r * self.t)))
        elif self.option_type == "p":
            return (np.multiply(norm.cdf(-d2) * self.K, np.exp(-self.r * self.t)) - \
        np.multiply(self.S,norm.cdf(-d1)))
        else:
            raise ValueError("specify the option type, c for call, p for put!")
    def calculate_vega(self):
        d1,d2 = self._calc_d()
        return self.S * norm.pdf(d1) * np.sqrt(self.t)



In [19]:
opt1 = Option(sigma=0.25,S=100,K=100,r=0.05,t=10/365)
print("theoretical option price is:",opt1.option_price())
print("vega on the option is:",opt1.calculate_vega())

theoretical option price is: 1.7189372043813407
vega on the option is: 6.593792028727019


In [ ]:
class IV:
    def __init__(self,ticker,option_type = "call",oi_lower_bound = 100):
        self.ticker = ticker
        self.option_type = option_type
        self.lower_bound = oi_lower_bound
    def 
    

    